<a href="https://colab.research.google.com/github/carlos-alves-one/-ML-Zoomcamp-HW-4/blob/main/ML_Zoomcamp_HW_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Machine Learning ZoomCamp - Homework 3 
**Author.....: Carlos Manuel de Oliveira Alves**<br>
**Created...: 02/10/2022**

In [29]:
# Import all necessay libraries that we will use in this homework
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
from sklearn.model_selection import KFold


from sklearn.metrics import mutual_info_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

from sklearn.linear_model import Ridge
from sklearn import metrics

# Import the library warnings to ignore the warnings from the system
import warnings
warnings.filterwarnings("ignore")

In [30]:
# Data Import:

# Locate dataset for the homework
data = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AER_credit_card_data.csv'

In [31]:
# Download dataset and store it locally with different file name
!wget $data -O data.csv

--2022-10-03 08:25:26--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AER_credit_card_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73250 (72K) [text/plain]
Saving to: ‘data.csv’

data.csv            100%[===================>]  71.53K  --.-KB/s    in 0.007s  

2022-10-03 08:25:26 (10.3 MB/s) - ‘data.csv’ saved [73250/73250]



In [32]:
# Store data using the Pandas library for this homework
data = pd.read_csv('data.csv')

In [33]:
# Check total of rows and columns of the dataset
print(data.shape)

(1319, 12)


In [34]:
# Check the first 5 rows of the dataset
data.head()

,card,reports,age,income,share,expenditure,owner,selfemp,dependents,months,majorcards,active
0,yes,0,37.66667,4.5200,0.033270,124.983300,yes,no,3,54,1,12
1,yes,0,33.25000,2.4200,0.005217,9.854167,no,no,3,34,1,13
2,yes,0,33.66667,4.5000,0.004156,15.000000,yes,no,4,58,1,5
3,yes,0,30.50000,2.5400,0.065214,137.869200,no,no,0,25,1,7
4,yes,0,32.16667,9.7867,0.067051,546.503300,yes,no,2,64,1,5


In [35]:
# Create the target variable by mapping yes to 1 and no to 0
data['card']=(data['card']=='yes').astype(int)

In [36]:
# Split the dataset into 3 parts: train/validation/test with 60%/20%/20% distribution. 
# Use train_test_split function for that with random_state=1
data_full_train,data_test=train_test_split(data,test_size=0.2,random_state=1)
data_train,data_val=train_test_split(data_full_train,test_size=0.25,random_state=1)

In [37]:
# Use the function info to get information about the dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1319 entries, 0 to 1318
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   card         1319 non-null   int64  
 1   reports      1319 non-null   int64  
 2   age          1319 non-null   float64
 3   income       1319 non-null   float64
 4   share        1319 non-null   float64
 5   expenditure  1319 non-null   float64
 6   owner        1319 non-null   object 
 7   selfemp      1319 non-null   object 
 8   dependents   1319 non-null   int64  
 9   months       1319 non-null   int64  
 10  majorcards   1319 non-null   int64  
 11  active       1319 non-null   int64  
dtypes: float64(4), int64(6), object(2)
memory usage: 123.8+ KB


In [38]:
# Get the card variables from train, validation and test dataframes
y_train = data_train.card.values
y_val   = data_val.card.values
y_test  = data_test.card.values

In [39]:
# Delete the serie card from the dataframes: train, validation and test
del data_train['card']
del data_val['card']
del data_test['card']

In [40]:
# Create an array with the numerical features
numerical_feature = data_train.select_dtypes(include=np.number).columns.tolist()
numerical_feature

['reports',
 'age',
 'income',
 'share',
 'expenditure',
 'dependents',
 'months',
 'majorcards',
 'active']

In [41]:
# Create an array with the categorical features
categorical =data.select_dtypes(include=['object']).columns.tolist()
categorical

['owner', 'selfemp']

In [45]:
# Create a list to store the scores of the ROC AUC
roc_auc=[]

# For loop to iterate all the numerical features
for feature in numerical_feature:

    # Create a model logistic regression
    model=LogisticRegression()

    # 
    model.fit(data_train[feature].to_frame(),y_train)
    y_pred=model.predict_proba(data_test[feature].to_frame())[:,1]
    fpr, tpr, thresholds = roc_curve(y_test, y_pred)
    score= auc(fpr, tpr)
    roc_auc.append([round(score,3),feature])
result = np.sort(roc_auc, axis = 0)

# Print the ROC AUC Scores by ascending order 
print("ROC AUC Scores:", result )

ROC AUC Scores: [['0.488' 'active']
 ['0.504' 'age']
 ['0.554' 'dependents']
 ['0.559' 'expenditure']
 ['0.583' 'income']
 ['0.605' 'majorcards']
 ['0.724' 'months']
 ['0.992' 'reports']
 ['0.997' 'share']]


In [ ]:
# Question 1 : Which numerical variable (among the following 4) has the highest AUC?
#    reports
#    dependents
#    active
#    share
#
# Answer for Question 1 : share